<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Literature_match_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd
import seaborn as sns

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [10]:
Auto_5_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_185_TF_corrected.csv')
Auto_4_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_172_TF_corrected.csv')
Auto_5_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_183_TF_corrected.csv', error_bad_lines=False)
Auto_4_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_179_TF_corrected.csv')
QC_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_PUF4_2_120k_159_TF_corrected.csv')

<ipython-input-10-878d1d7399cd>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_5_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_183_TF_corrected.csv', error_bad_lines=False)
Skipping line 121: expected 21 fields, saw 23



In [11]:
Auto_5_gp_df['Ion Type'] = 'gp'
Auto_4_gp_df['Ion Type'] = 'gp'
Auto_5_pm_df['Ion Type'] = 'gp'
Auto_4_pm_df['Ion Type'] = 'gp'
Auto_5_gp_df['Sample Name'] = 'Auto_5'
Auto_4_gp_df['Sample Name'] = 'Auto_4'
Auto_5_pm_df['Sample Name'] = 'Auto_5'
Auto_4_pm_df['Sample Name'] = 'Auto_4'

In [12]:
print(Auto_5_gp_df.columns)

Index(['Component Name', 'Retention Time', 'Reference m/z', 'Area', 'Height',
       'TIC', 'Formula (mol ion)', 'CAS No.', 'SI', 'RSI', 'HRF Score',
       'RHRF Score', 'Total Score', 'Selected Column Type', 'Calculated RI',
       'Library RI', 'Delta RI', 'Library Name', 'Library ID Number',
       'Sample Name', 'File name', 'Ion Type'],
      dtype='object')


In [24]:
print('Auto5_gp:', Auto_5_gp_df.shape)
print('Auto4_gp:', Auto_4_gp_df.shape)
print('Auto5_pm:', Auto_5_pm_df.shape)
print('Auto4_pm:', Auto_4_pm_df.shape)

Auto5_gp: (259, 23)
Auto4_gp: (465, 23)
Auto5_pm: (182, 23)
Auto4_pm: (197, 23)


In [13]:
def area_ratio(df):
    """
    Calculates ratio of area to the internal standard and saves it in a new column
    """
    benzyl_benzoate_area = df.loc[df['Component Name'] == 'Benzyl Benzoate', 'Area'].values[0]
    df['Area Ratio'] = df['Area'] / benzyl_benzoate_area

    return df

def filter_dataframe(df):
    """
    Removing the results below the probability threshold that is set below
    """
    condition1 = df['RHRF Score'] >= 90
    condition2 = df['RSI'] >= 600
    condition3 = (df['Total Score'] >= 70) & ~(condition1 & condition2)

    filtered_df = df[condition1 & condition2 | condition3]

    return filtered_df

def rearrange_by_area_ratio(df):
    """
    Rearrearanging the rows in descending fashion from the peak area ratio
    """
    if 'Area Ratio' not in df.columns:
        raise ValueError("DataFrame must have a 'Area Ratio' column")

    # Sort the DataFrame based on the 'Area Ratio' column in descending order
    df_sorted = df.sort_values(by='Area Ratio', ascending=False)

    # Reset the index to reflect the new order
    df_sorted = df_sorted.reset_index(drop=True)

    return df_sorted

def qc_correction(sample_df, ref_df, threshold):
    """
    Function removing the rows that are in the blank under the condition given
    """
    result_df = pd.DataFrame(columns=sample_df.columns)
    matched_indices = set()

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df, threshold)
        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)
            if best_match is not None and best_match['Area Ratio'] >= 0.05:
                matched_indices.add(best_match.name)

    result_df = sample_df[~sample_df.index.isin(matched_indices)]
    return result_df

def find_comon_row(sample_df, ref_df, threshold):
    """
    Function showing what compounds were in the sample and blank that matched
    """
    result_df = pd.DataFrame(columns=sample_df.columns)

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df, threshold)
        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)
            if best_match is not None and best_match['Area Ratio'] >= 0.05:
                result_df = result_df.append(sample_row, ignore_index=True)

    return result_df

def find_matching_rows(sample_row, ref_df, threshold):
    """
    Retention time threshold matching
    """
    matching_rows = ref_df[
        (ref_df['Retention Time'] >= sample_row['Retention Time'] - threshold) &
        (ref_df['Retention Time'] <= sample_row['Retention Time'] + threshold)
    ]
    return matching_rows

def find_best_match(sample_row, matching_rows):
    """
    Compound comparison for the exclusion parameters
    """
    best_match = None
    min_diff = float('inf')

    for _, ref_row in matching_rows.iterrows():
        diff = compare_strings(sample_row['Formula (mol ion)'], ref_row['Formula (mol ion)'])
        if diff < min_diff:
            min_diff = diff
            best_match = ref_row

    return best_match

def compare_strings(str1, str2):
    # Implement your string comparison logic here
    # You can use fuzzy string matching or any other suitable method
    # For simplicity, let's assume an exact match for now
    return 0 if str1 == str2 else 1

In [14]:
Auto_5_gp_df = area_ratio(Auto_5_gp_df)
Auto_4_gp_df = area_ratio(Auto_4_gp_df)
Auto_5_pm_df = area_ratio(Auto_5_pm_df)
Auto_4_pm_df = area_ratio(Auto_4_pm_df)
QC_df = area_ratio(QC_df)

In [23]:
threshold_value = 0.05 # Set your desired threshold value

Auto_5_gp_df_filtered = qc_correction(Auto_5_gp_df, QC_df, threshold_value)
Auto_4_gp_df_filtered = qc_correction(Auto_4_gp_df, QC_df, threshold_value)
Auto_5_pm_df_filtered = qc_correction(Auto_5_pm_df, QC_df, threshold_value)
Auto_4_pm_df_filtered = qc_correction(Auto_4_pm_df, QC_df, threshold_value)
print('Auto5_gp:', Auto_5_gp_df_filtered.shape)
print('Auto4_gp:', Auto_4_gp_df_filtered.shape)
print('Auto5_pm:', Auto_5_pm_df_filtered.shape)
print('Auto4_pm:', Auto_4_pm_df_filtered.shape)
#Auto_5_gp_df_filtered.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_corrected.csv')

Auto5_gp: (245, 23)
Auto4_gp: (428, 23)
Auto5_pm: (175, 23)
Auto4_pm: (187, 23)


In [26]:
Auto5_gp_comon_df = find_comon_row(Auto_5_gp_df, QC_df, threshold_value)
Auto4_gp_comon_df = find_comon_row(Auto_4_gp_df, QC_df, threshold_value)
Auto5_pm_comon_df = find_comon_row(Auto_5_pm_df, QC_df, threshold_value)
Auto4_pm_comon_df = find_comon_row(Auto_4_pm_df, QC_df, threshold_value)
print('Auto5_gp:', Auto5_gp_comon_df.shape)
print('Auto4_gp:', Auto4_gp_comon_df.shape)
print('Auto5_pm:', Auto5_pm_comon_df.shape)
print('Auto4_pm:', Auto4_pm_comon_df.shape)
#Auto5_gp_comon_df.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_matches.csv')

<ipython-input-13-e0667a905259>:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-13-e0667a905259>:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-13-e0667a905259>:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-13-e0667a905259>:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)


Auto5_gp: (37, 23)
Auto4_gp: (50, 23)
Auto5_pm: (23, 23)
Auto4_pm: (27, 23)


In [28]:
Auto5_gp_ordered = rearrange_by_area_ratio(Auto_5_gp_df_filtered)
Auto4_gp_ordered = rearrange_by_area_ratio(Auto_4_gp_df_filtered)
Auto5_pm_ordered = rearrange_by_area_ratio(Auto_5_pm_df_filtered)
Auto4_pm_ordered = rearrange_by_area_ratio(Auto_4_pm_df_filtered)
print(Auto5_gp_ordered.head())
print(Auto4_gp_ordered.head())
print(Auto5_pm_ordered.head())
print(Auto4_pm_ordered.head())
#Auto5_gp_ordered.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_rearranged.csv')

                                      Component Name  Retention Time  \
0                           Butylated Hydroxytoluene          13.668   
1  Phenol, 2,4-bis(1,1-dimethylethyl)-, phosphite...          31.184   
2                            2,4-Di-tert-butylphenol          13.719   
3  Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
4                 Carbonic acid, undecyl vinyl ester          11.236   

   Reference m/z         Area      Height          TIC Formula (mol ion)  \
0     205.158646  18178762411  3304219877  10312517507           C15H24O   
1     441.291504   7606397204   695700108   2525748710         C42H63O3P   
2     191.142838   4936204315   858168495   1174362456           C14H22O   
3     219.174301   3487699203   201822395   2219112596          C35H62O3   
4      43.054165   1854075664   323340139   2128567954          C14H26O3   

      CAS No.   SI  RSI  ...  Selected Column Type  Calculated RI  Library RI  \
0    128-37-0  863  864  ... 

In [ ]:
filtered_df = filter_dataframe(rearranged_df)
print(filtered_df)
filtered_df.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_filtered.csv')

                                        Component Name  Retention Time  \
0                             Butylated Hydroxytoluene          13.668   
1    Phenol, 2,4-bis(1,1-dimethylethyl)-, phosphite...          31.184   
2                              2,4-Di-tert-butylphenol          13.719   
3    Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
4                   Carbonic acid, undecyl vinyl ester          11.236   
..                                                 ...             ...   
240  Methyl 2-ethyl-1,2,3,4-tetrahydro-.alpha.-(met...          23.218   
241                                       Thioxanthene          20.189   
242                        Phenanthrene, 3,6-dimethyl-          22.455   
243                 cyclohexane, [1,1'-biphenyl]-4-yl-          22.497   
244  Glycyl-L-valine, N-dimethylaminomethylene-, me...          24.327   

     Reference m/z         Area      Height          TIC Formula (mol ion)  \
0       205.158646  18178762411  